<a href="https://colab.research.google.com/github/Yundo37/automation-portfolio-Yundo/blob/main/Test_Automation_Yundo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update -y
!apt-get install -y chromium-browser chromium-chromedriver
print("✅ Chrome 설치 완료!")


In [ ]:
# GitHub 레포 클론
!git clone https://github.com/Yundo37/automation-portfolio-Yundo.git workdir

# 작업 디렉토리로 이동
%cd workdir

# 필요한 라이브러리 설치
!pip install selenium pytest pytest-html

print("✅ 레포 클론 및 라이브러리 설치 완료!")

In [ ]:
# 파일 구조 확인
!ls -la
print("현재 디렉토리:", %pwd)


In [ ]:
# 테스트 실행
!pytest -v --tb=short --html=report.html --self-contained-html --browser_name=chrome

print("✅ 테스트 실행 완료!")


In [ ]:
# 리포트 파일 존재 확인
!ls -la *.html

# 리포트 내용 Colab에서 바로 보기
try:
    from IPython.display import HTML
    with open('report.html', 'r', encoding='utf-8') as f:
        display(HTML(f.read()))
    print("✅ 리포트 로딩 완료!")
except FileNotFoundError:
    print("❌ report.html 파일을 찾을 수 없습니다.")
    print("pytest 실행이 성공했는지 확인해주세요.")